# Main

## Referencias
https://lotterycodex.com/lottery-formula/
https://pypi.org/project/loteria-caixa/#description

## Objetivos
[x] Criar um gerador de apostas aleatorias
[x] Criar um gerador de combinacoes possiveis. Ex. Todas as combinacoes possiveis de 4 dezenas dentro dessas 6 dezenas
[] Eliminar as combinacoes repetidas
[x] Fazer um algoritmo de geração de numeros que considere pares e impares, altos e baixos
[x] Pegar todos os resultados da lotofacil
[] Gerar um calculador de resultados
[] Testar as hipoteses usando os resultados anteriores
[] Imprimir as apostas que devem ser feitas 

In [55]:
# Imports

import random
from itertools import combinations
from loteria_caixa import *
import pandas as pd

In [54]:
# Funcoes base

def do_generate_bet(dezenas, intervalo=100):
    return sorted(random.sample(range(1, intervalo+1), dezenas))

def do_calculate_combinations(dezenas, grupamento):
    return tuple(sorted(combo) for combo in combinations(dezenas, grupamento))

def do_calculate_evens_odds(numeros):
    saida = {
        'pares': 0,
        'impares': 0,
        'diferenca': 0
    }

    for numero in numeros:
        if numero % 2 == 0:
            saida['pares'] += 1
        else:
            saida['impares'] +=1
    
    saida['diferenca'] = abs(saida['pares'] - saida['impares'])
    return saida

def do_calculate_high_low_value(numeros, intervalo=100):
    saida = {
        'high': 0,
        'low': 0,
        'diferenca': 0
    }

    media = do_calculate_range_average(range(1, intervalo + 1))

    for numero in numeros:
        if numero >= media:
            saida['high'] += 1
        else:
            saida['low'] += 1
    saida['diferenca'] = abs(saida['high'] - saida['low'])
    
    return saida

def do_calculate_range_average(intervalo):
    if len(intervalo) == 0:
        return None
    
    return round(sum(intervalo)/len(intervalo))

def contar_itens(lista_a, lista_b):
    saida = 0
    for item in lista_a:
        if item in lista_b:
            saida += 1
    return saida

def verificar_existencia(lista, tupla):
    for sublista in tupla:
        if set(lista).issubset(sublista):
            return True
    return False


In [61]:
#gerar apostas

def generate_bets(quant_numbers=6, range_numbers=60, quant_bets=1, even_odds=False, even_odds_dif=1, high_low=False, high_low_dif=1):

    settings = {
        'quant_numbers': quant_numbers,
        'range_numbers': range_numbers,
        'quant_bets': quant_bets,
        'even_odds': { 'evaluate': even_odds, 'number_dif': even_odds_dif},
        'high_low': { 'evaluate': high_low, 'number_dif': high_low_dif},
    }

    saida = {
        'settings': settings,
        'bets': []
    }

    while len(saida['bets']) < settings['quant_bets']:
        aposta = do_generate_bet(settings['quant_numbers'], settings['range_numbers'])
        even_odds_result = do_calculate_evens_odds(aposta)
        high_low_result = do_calculate_high_low_value(aposta, settings['range_numbers'])

        dados_aposta = {
            'bet': aposta,
            'even_odds': even_odds_result,
            'high_low': high_low_result
        }

        if(settings['even_odds']['evaluate'] and not(settings['even_odds']['number_dif'] >= even_odds_result['diferenca'])):
            #pular_execucao
            continue

        if(settings['high_low']['evaluate'] and not(settings['high_low']['number_dif'] >= high_low_result['diferenca'])):
            #pular_execucao
            continue

        saida['bets'].append(dados_aposta)
    
    return saida

teste = generate_bets(quant_bets=1, quant_numbers=15, range_numbers=25, even_odds=True, high_low=True)

teste

{'settings': {'quant_numbers': 15,
  'range_numbers': 25,
  'quant_bets': 1,
  'even_odds': {'evaluate': True, 'number_dif': 1},
  'high_low': {'evaluate': True, 'number_dif': 1}},
 'bets': [{'bet': [1, 2, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 23],
   'even_odds': {'pares': 8, 'impares': 7, 'diferenca': 1},
   'high_low': {'high': 8, 'low': 7, 'diferenca': 1}}]}

In [4]:
aposta

[2, 5, 6, 7, 8, 9, 10, 14, 15, 18, 19, 22, 23, 24, 25]

In [5]:

df = pd.read_csv('resultados_concursos\\resultados_lotofacil.csv')



df['dezenas'] = df['dezenas'].str.split(',')

# Aplicando uma função para formatar cada string com colchetes
df['dezenas'] = df['dezenas'].apply(lambda x: [int(item) for item in x])
print(df)
#for aposta in apostas:


      sorteio                                            dezenas
0        3055  [2, 3, 7, 8, 9, 10, 11, 12, 14, 15, 17, 20, 21...
1        3054  [1, 2, 4, 8, 9, 10, 14, 15, 18, 19, 20, 21, 22...
2        3053  [2, 4, 5, 7, 8, 12, 13, 14, 16, 17, 18, 19, 21...
3        3052  [2, 3, 4, 8, 9, 10, 11, 14, 15, 18, 19, 20, 21...
4        3051  [2, 3, 4, 6, 9, 10, 11, 12, 15, 17, 18, 20, 21...
...       ...                                                ...
3050        5  [1, 2, 4, 8, 9, 11, 12, 13, 15, 16, 19, 20, 23...
3051        4  [1, 2, 4, 5, 8, 10, 12, 13, 16, 17, 18, 19, 23...
3052        3  [1, 4, 6, 7, 8, 9, 10, 11, 12, 14, 16, 17, 20,...
3053        2  [1, 4, 5, 6, 7, 9, 11, 12, 13, 15, 16, 19, 20,...
3054        1  [2, 3, 5, 6, 9, 10, 11, 13, 14, 16, 18, 20, 23...

[3055 rows x 2 columns]


In [6]:
print(apostas[0])
print(df['dezenas'].iloc[0])

resultado = contar_itens(apostas[0], df['dezenas'].iloc[0])

resultado

[1, 2, 3, 5, 6, 9, 10, 12, 13, 15, 20, 22, 23, 24, 25]
[2, 3, 7, 8, 9, 10, 11, 12, 14, 15, 17, 20, 21, 22, 23]


9

In [43]:
apostas = []

while len(apostas) < 10:
    aposta = do_generate_bet(15, 25)
    par_impar = do_calculate_evens_odds(aposta)
    high_low = do_calculate_high_low_value(aposta, 25)

    if par_impar['diferenca'] < 2 and high_low['diferenca'] < 2:
        apostas.append(aposta)

estatisticas_apostas = []

for aposta in apostas:
    saida = {
        'aposta': aposta,
        'premiado': 0,
        'nao_premiado': 0,
        '< 11': 0,
        '11': 0,
        '12': 0,
        '13': 0,
        '14': 0,
        '15': 0
    }

    for index, row in df.iterrows():
        comparacao = contar_itens(aposta, row['dezenas'])
        if comparacao > 10:
            saida[str(comparacao)] += 1
            saida['premiado'] += 1
        else:
            saida['< 11'] += 1
            saida['nao_premiado'] += 1

    estatisticas_apostas.append(saida)

for resultado in estatisticas_apostas:
    print(resultado)

{'aposta': [1, 2, 3, 4, 5, 7, 8, 10, 13, 14, 16, 18, 22, 23, 25], 'premiado': 334, 'nao_premiado': 2721, '< 11': 2721, '11': 275, '12': 52, '13': 7, '14': 0, '15': 0}
{'aposta': [3, 4, 5, 6, 7, 8, 9, 10, 15, 16, 18, 19, 20, 21, 25], 'premiado': 321, 'nao_premiado': 2734, '< 11': 2734, '11': 252, '12': 59, '13': 10, '14': 0, '15': 0}
{'aposta': [1, 2, 3, 6, 10, 11, 12, 13, 14, 15, 16, 17, 18, 24, 25], 'premiado': 307, 'nao_premiado': 2748, '< 11': 2748, '11': 248, '12': 57, '13': 2, '14': 0, '15': 0}
{'aposta': [1, 3, 4, 8, 9, 10, 12, 16, 18, 19, 20, 21, 23, 24, 25], 'premiado': 330, 'nao_premiado': 2725, '< 11': 2725, '11': 276, '12': 49, '13': 5, '14': 0, '15': 0}
{'aposta': [1, 2, 3, 4, 6, 7, 8, 14, 15, 16, 17, 20, 21, 22, 25], 'premiado': 304, 'nao_premiado': 2751, '< 11': 2751, '11': 253, '12': 48, '13': 3, '14': 0, '15': 0}
{'aposta': [2, 4, 5, 6, 8, 9, 12, 13, 15, 19, 20, 21, 22, 23, 24], 'premiado': 321, 'nao_premiado': 2734, '< 11': 2734, '11': 259, '12': 53, '13': 9, '14': 0, 

In [19]:
estatisticas_apostas = []
resultado_caixa = [[4, 3, 8, 23, 11, 24, 21, 13, 19, 18, 22, 16, 1, 15, 20]]

par_impar = do_calculate_evens_odds(resultado_caixa[0])
high_low = do_calculate_high_low_value(resultado_caixa[0], 25)

print('metricas sorteado: par_impar - ', par_impar, ' high_low: ', high_low)

for aposta in apostas:
    saida = {
        'aposta': aposta,
        'acertos': 0,
        'premiado': 0,
        'nao_premiado': 0,
        '< 11': 0,
        '11': 0,
        '12': 0,
        '13': 0,
        '14': 0,
        '15': 0
    }

    for row in resultado_caixa:
        comparacao = contar_itens(aposta, row)
        saida['acertos'] = comparacao
        if comparacao > 10:
            saida[str(comparacao)] += 1
            saida['premiado'] += 1
        else:
            saida['< 11'] += 1
            saida['nao_premiado'] += 1

    estatisticas_apostas.append(saida)

for resultado in estatisticas_apostas:
    print(resultado)

metricas sorteado: par_impar -  {'pares': 7, 'impares': 8, 'diferenca': 1}  high_low:  {'high': 10, 'low': 5, 'diferenca': 5}
{'aposta': [1, 2, 3, 5, 6, 9, 10, 12, 13, 15, 20, 22, 23, 24, 25], 'acertos': 8, 'premiado': 0, 'nao_premiado': 1, '< 11': 1, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0}
{'aposta': [1, 2, 3, 5, 6, 8, 9, 14, 15, 18, 19, 20, 23, 24, 25], 'acertos': 9, 'premiado': 0, 'nao_premiado': 1, '< 11': 1, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0}
{'aposta': [1, 2, 4, 5, 6, 8, 11, 13, 14, 16, 20, 21, 23, 24, 25], 'acertos': 10, 'premiado': 0, 'nao_premiado': 1, '< 11': 1, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0}
{'aposta': [1, 2, 3, 5, 7, 9, 10, 12, 18, 19, 20, 21, 22, 24, 25], 'acertos': 8, 'premiado': 0, 'nao_premiado': 1, '< 11': 1, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0}
{'aposta': [1, 2, 5, 6, 8, 9, 11, 13, 14, 15, 16, 18, 23, 24, 25], 'acertos': 9, 'premiado': 0, 'nao_premiado': 1, '< 11': 1, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0}
{'aposta': [1, 2, 3

In [ ]:
apostas = combinacoes_possiveis

for quina in combinacoes_quina

In [173]:
apostas = []
# processamento das quinas
for quina in combinacoes_quina:
    print('------------nova iteracao quina --------------')
    ja_contemplada = False
    #checa se a quina já está contemplada na lista
    for aposta in apostas:
        if set(quina).issubset(aposta):
            ja_contemplada = True
            
    
    # nao foi contemplada, vamos iterar pelas senas e adicionar a primeira que servir
    print('apostas: ', apostas)
    print('quina: ', quina)
    print('ja contemplada: ', ja_contemplada)
    if(not ja_contemplada):
        ja_adicionada = False
        for sena in combinacoes_sena:
            if(set(quina).issubset(sena) and not ja_adicionada ):
                apostas.append(sena)
                ja_adicionada = True
    print('Fim da iteracao. Apostas atuais: ', apostas)
    
len(apostas)

------------nova iteracao quina --------------
apostas:  []
quina:  [2, 13, 21, 35, 39]
ja contemplada:  False
Fim da iteracao. Apostas atuais:  [[2, 13, 21, 35, 39, 47]]
------------nova iteracao quina --------------
apostas:  [[2, 13, 21, 35, 39, 47]]
quina:  [2, 13, 21, 35, 47]
ja contemplada:  True
Fim da iteracao. Apostas atuais:  [[2, 13, 21, 35, 39, 47]]
------------nova iteracao quina --------------
apostas:  [[2, 13, 21, 35, 39, 47]]
quina:  [2, 13, 21, 35, 51]
ja contemplada:  False
Fim da iteracao. Apostas atuais:  [[2, 13, 21, 35, 39, 47], [2, 13, 21, 35, 39, 51]]
------------nova iteracao quina --------------
apostas:  [[2, 13, 21, 35, 39, 47], [2, 13, 21, 35, 39, 51]]
quina:  [2, 13, 21, 39, 47]
ja contemplada:  True
Fim da iteracao. Apostas atuais:  [[2, 13, 21, 35, 39, 47], [2, 13, 21, 35, 39, 51]]
------------nova iteracao quina --------------
apostas:  [[2, 13, 21, 35, 39, 47], [2, 13, 21, 35, 39, 51]]
quina:  [2, 13, 21, 39, 51]
ja contemplada:  True
Fim da iteracao.

6

In [163]:
# Conjunto de números fornecido
numbers = [2, 13, 21, 35, 39, 47, 51]

# Função para verificar se uma combinação cobre todos os arranjos de quatro números
def cobre_arranjos(comb):
    arranjos = set(combinations(comb, 4))
    for arranjo in arranjos:
        if arranjo not in set(combinations(numbers, 4)):
            return False
    return True

# Gerar todas as combinações de seis números
comb_6 = combinations(numbers, 6)

# Verificar cada combinação de seis números
arranjos_minimos = []
for comb in comb_6:
    if cobre_arranjos(comb):
        arranjos_minimos.append(comb)

# Imprimir os arranjos mínimos encontrados
for arranjo in arranjos_minimos:
    print(', '.join(map(str, arranjo)))

2, 13, 21, 35, 39, 47
2, 13, 21, 35, 39, 51
2, 13, 21, 35, 47, 51
2, 13, 21, 39, 47, 51
2, 13, 35, 39, 47, 51
2, 21, 35, 39, 47, 51
13, 21, 35, 39, 47, 51


In [ ]:


def generate_combinations(numbers, k):
    return list(combinations(numbers, k))

def generate_mega_sena_combinations(fechamento):
    all_numbers = list(range(1, 61))  # Mega Sena vai de 1 a 60
    combinations_to_play = []

    for num, quantity in fechamento:
        combinations_to_play.extend(generate_combinations(all_numbers, num) * quantity)

    return combinations_to_play

def main():
    fechamento = [(6, 10), (1, 12), (2, 12)]  # Exemplo de fechamento: jogar 6 números em 10 combinações, 1 número em 12 combinações, 2 números em 12 combinações
    combinations = generate_mega_sena_combinations(fechamento)
    
    print(f"Total de combinações a jogar: {len(combinations)}")
    # Aqui você pode imprimir ou manipular as combinações geradas conforme necessário

if __name__ == "__main__":
    main()


TypeError: 'tuple' object cannot be interpreted as an integer

In [1]:
import site
print(site.getsitepackages())

['c:\\Program Files\\Python312', 'c:\\Program Files\\Python312\\Lib\\site-packages']
